# Elastodynamics using the Finite Elements Method (FEM)


## Problem statement


The one-dimensional elastic wave equation is given by:

$$
\rho \frac{\partial^2 u}{\partial t^2} = \frac{\partial}{\partial x} \mu \frac{\partial u}{\partial x} + f
$$


The corresponding weak form is:

$$
\int_{D} \rho  \frac{\partial^{2} \overline{u}}{\partial t^{2}} \varphi_{j} dx + \int_{D} \mu \frac{\partial \overline{u}}{\partial x} \varphi_{j} dx = \int_{D} f \varphi_{j} dx 
$$

which can be expressed in matrix form as:

$$
\mathbf{M}^{T} \frac{\partial^{2}}{\partial t^{2}} \mathbf{u}  + \mathbf{K}^{T} \mathbf{u}  = \mathbf{f}
$$

where $\mathbf{M}$ is the mass matrix, $\mathbf{K}$ is the stiffness matrix, $\mathbf{f}$ the source vector and $\mathbf{u}$ the displacements vector. Considering the approximation:

$$
\frac{\partial^{2}}{\partial t^{2}} \mathbf{u} \approx \frac{u(t+dt)-2u(t)+u(t-dt)}{dt^{2}}
$$

we get that:

$$
\mathbf{u}(t+dt) = dt^{2} \left( \mathbf{M}^{T} \right)^{-1} \left[ \mathbf{f} - \mathbf{K}^{T} \mathbf{u} \right] + 2 \mathbf{u}(t) + \mathbf{u}(t + dt) 
$$


## Import required libraries

In [8]:
from mpi4py import MPI
from dolfinx import mesh
domain = mesh.create_unit_square(MPI.COMM_WORLD, 8, 8, mesh.CellType.quadrilateral)

## Functions

In [12]:
from dolfinx.fem import FunctionSpace
V = FunctionSpace(domain, ("Lagrange", 1))
from dolfinx import fem
uD = fem.Function(V)
uD.interpolate(lambda x: 1 + x[0]**2 + 2 * x[1]**2)
import numpy
# Create facet to cell connectivity required to determine boundary facets
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)
boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
bc = fem.dirichletbc(uD, boundary_dofs)

In [13]:
import ufl
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

In [14]:
from dolfinx import default_scalar_type
f = fem.Constant(domain, default_scalar_type(-6))

In [15]:
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx

In [16]:
from dolfinx.fem.petsc import LinearProblem
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [18]:
import pyvista
print(pyvista.global_theme.jupyter_backend)

trame


In [19]:
from dolfinx import plot
pyvista.start_xvfb()
topology, cell_types, geometry = plot.vtk_mesh(domain, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

OSError: Please install Xvfb with:

Debian
$ sudo apt install libgl1-mesa-glx xvfb

CentOS / RHL
$ sudo yum install libgl1-mesa-glx xvfb



In [20]:
import matplotlib.pyplot as plt
import pyvista
import ufl
import numpy as np

from petsc4py import PETSc
from mpi4py import MPI

from dolfinx import fem, mesh, io, plot
from dolfinx.fem.petsc import assemble_vector, assemble_matrix, create_vector, apply_lifting, set_bc

# Define temporal parameters
t = 0  # Start time
T = 1.0  # Final time
num_steps = 50
dt = T / num_steps  # time step size

# Define mesh
nx, ny = 50, 50
domain = mesh.create_rectangle(MPI.COMM_WORLD, [np.array([-2, -2]), np.array([2, 2])],
                               [nx, ny], mesh.CellType.triangle)
V = fem.FunctionSpace(domain, ("Lagrange", 1))

In [21]:
# Create initial condition
def initial_condition(x, a=5):
    return np.exp(-a * (x[0]**2 + x[1]**2))


u_n = fem.Function(V)
u_n.name = "u_n"
u_n.interpolate(initial_condition)

# Create boundary condition
fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(
    domain, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))
bc = fem.dirichletbc(PETSc.ScalarType(0), fem.locate_dofs_topological(V, fdim, boundary_facets), V)

In [22]:
xdmf = io.XDMFFile(domain.comm, "diffusion.xdmf", "w")
xdmf.write_mesh(domain)

# Define solution variable, and interpolate initial solution for visualization in Paraview
uh = fem.Function(V)
uh.name = "uh"
uh.interpolate(initial_condition)
xdmf.write_function(uh, t)

In [23]:
u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
f = fem.Constant(domain, PETSc.ScalarType(0))
a = u * v * ufl.dx + dt * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = (u_n + dt * f) * v * ufl.dx

In [24]:
bilinear_form = fem.form(a)
linear_form = fem.form(L)

In [27]:
A = assemble_matrix(bilinear_form, bcs=[bc])
A.assemble()
b = create_vector(linear_form)

In [28]:
solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

In [29]:
import matplotlib as mpl
pyvista.start_xvfb()

grid = pyvista.UnstructuredGrid(*plot.vtk_mesh(V))

plotter = pyvista.Plotter()
plotter.open_gif("u_time.gif", fps=10)

grid.point_data["uh"] = uh.x.array
warped = grid.warp_by_scalar("uh", factor=1)

viridis = mpl.colormaps.get_cmap("viridis").resampled(25)
sargs = dict(title_font_size=25, label_font_size=20, fmt="%.2e", color="black",
             position_x=0.1, position_y=0.8, width=0.8, height=0.1)

renderer = plotter.add_mesh(warped, show_edges=True, lighting=False,
                            cmap=viridis, scalar_bar_args=sargs,
                            clim=[0, max(uh.x.array)])

OSError: Please install Xvfb with:

Debian
$ sudo apt install libgl1-mesa-glx xvfb

CentOS / RHL
$ sudo yum install libgl1-mesa-glx xvfb



## Style for text in Markdown

In [3]:
from IPython.core.display import HTML
def css_styling():
    styles = open('estilo.css', 'r').read()
    return HTML(styles)
css_styling()